# Exercise 6 - SIRV Models - Example Solution

In [ ]:
import Pkg; 
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Nextcloud/Exercise6.jl`
┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.9.2). Unexpected behavior may occur.
└ @ nothing /Users/max/Nextcloud/Exercise6.jl/Manifest.toml:0
   Installed Multisets ───────────── v0.4.4
   Installed ImageIO ─────────────── v0.6.7
   Installed StableHashTraits ────── v1.1.1
   Installed Permutations ────────── v0.4.19
   Installed JpegTurbo ───────────── v0.1.4
   Installed TiffImages ──────────── v0.6.8
   Installed DifferentialEquations ─ v7.11.0
   Installed PNGFiles ────────────── v0.4.2
   Installed Sundials_jll ────────── v5.2.1+0
   Installed Mods ────────────────── v1.3.3
   Installed SteadyStateDiffEq ───── v1.16.1
   Installed BoundaryValueDiffEq ─── v5.4.0
   Installed AbstractLattices ────── v0.3.0
   Installed SimpleRandom ────────── v0.3.1
   Installed SimplePartitions ────── v0.3.1
   Installed SciMLBase ───────────── v2.9.1
   Installed GridLayoutBase ───────

In [ ]:
using Exercise6

- In this notebook we'll use our package [Exercise6.jl](https://github.com/maximilian-gelbrecht/Exercise6.jl) to explore the dynamics of various SIR models modified to include the effect of vaccination. Remember that there is no single correct solution to this exercise; the aim was simply for you to explore some variations of an SIR model while also gaining experience structuring your code as a proper Julia package. What follows is therefore only one possible approach that could have been taken.


- While using this notebook, it is highly recommend to look closely at the code and the documentation for the package [Exercise6.jl](https://github.com/maximilian-gelbrecht/Exercise6.jl) where the models and plots are implemented.


- When using this notebook remember also that you can inspect the docstrings for any Julia object using `?`. For example:

In [ ]:
?SIRV

- We will use the same initial conditions throughout: 90% of the population susceptible, 5% infected, 5% recovered, and 0% vaccinated.

In [ ]:
u0 = [0.9, 0.05, 0.05, 0.0]

- In addition, we'll stick with the same default parameters for the contact rate $\beta = 0.35$ and recovery rate $\gamma = 0.035$ taken from Fig. 1 of [this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7321055/pdf/main.pdf).


- Let's start off by looking at the trajectory of a standard SIRV model.

In [ ]:
ν = 0.01  # Fraction of the susceptible population vaccinated per day
model = SIRV(ν)
plot(model; u0)

- If we vaccinate 1% of the susceptible population every day, we eventually end up with no infections (remember that the vaccine is assumed 100% effective), but over 90% of the population got their immunity from infection rather than vaccination.


- How about if we could vaccinate 5% of the susceptible population every day?

In [ ]:
ν = 0.05  # Fraction of the susceptible population vaccinated per day
model = SIRV(ν)
plot(model; u0)

- This is better, with 39% of the population now avoiding infection.


- How fast do we have to vaccinate people in order to beat the pace of infections?

In [ ]:
ν_range = 0.0:0.01:0.5
plot_total_infections_by_vax_rate(ν_range; u0)

- We see that, even for very high vaccination rates, a significant proportion of the population still becomes infected before they can get vaccinated.


- Now let's look at what happens when the immunity obtained from both infection and vaccination decays over time. This is implemented by the type `SIRVDecayingImmunity`.

In [ ]:
ν = 0.01     # 1% of the susceptible population vaccinated per day
α = 2 / 365  # Immunity from infection decays after half a year, on average
μ = 2 / 365  # Immunity from vaccination decays after half a year, on average
model = SIRVDecayingImmunity(ν, α, μ)
plot(model; u0, endtime = 2 * 365.0)

- For these parameters, the model has a fixed point where 10% of the population is infected at any given time.


- How does the fixed point depend on the vaccination rate $\nu$?

In [ ]:
ν = 0.02
model = SIRVDecayingImmunity(ν, α, μ)
plot(model; u0)

- So the endemic equilibrium point depends on the vaccination rate, as we might expect.


- How fast do we have to vaccinate people for the stable fixed point of the model to be a disease-free equilibrium rather than an endemic equilibrium?

In [ ]:
ν = 0.065
model = SIRVDecayingImmunity(ν, α, μ)
plot(model; u0)

- For the disease to go extinct, we have to vaccinate about 6.5% of the susceptible population every day!


- Now let's see what happens when we additionally introduce some seasonality into the contact rate $\beta$.

In [ ]:
β₁ = 0.1     # Max 10% variation around the average contact rate
ν = 0.01     # 1% of the susceptible population vaccinated per day
α = 2 / 365  # Immunity from infection decays after half a year, on average
μ = 2 / 365  # Immunity from vaccination decays after half a year, on average

model = SIRVSeasonalContactDecayingImmunity(β₁, ν, α, μ)
plot(model; u0, endtime = 5 * 365.0)

- Now we observe oscillations in the infection level, with around 10% of the population being infected at any given time.


- Let's look at this in phase space.

In [ ]:
plot_phase_diagram(model, endtime = 2 * 365.0, Ttr = 365.0)

- Can we observe any bifurcations, like we did in the lecture?

In [ ]:
β₁ = 0.9
model = SIRVSeasonalContactDecayingImmunity(β₁, ν, α, μ)
plot_phase_diagram(model, endtime = 2 * 365.0, Ttr = 365.0)

- While the dynamics begin to look interesting for large (perhaps unrealistic) values of β₁, with the peaks and troughs also becoming more severe, we nonetheless consistently observe the same annual periodicity as the forcing. That is, we don't see any period doubling bifurcations or chaos, like we did for a similar model in the lecture. Interesting!